# Finalise facebook comments topics
Finalises the facebook comments topics.
And persists it to s3.
See docs for more information.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import pandas as pd
import tentaclio

from phoenix.common import artifacts, run_params
from phoenix.common import utils
from phoenix.tag import export
from phoenix.tag import finalise
from phoenix.tag import object_filters

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parameters
# See phoenix/common/run_datetime.py expected format of parameter
RUN_DATETIME = None
TENANT_ID = None

# See phoenix/common/artifacts/registry_environment.py expected format of parameter
ARTIFACTS_ENVIRONMENT_KEY = "local"

# Filters for batch
YEAR_FILTER = 2021
# Without zero padding
MONTH_FILTER = 8
OBJECT_TYPE = "facebook_comments"

# OUTPUT
FINAL_URL_FACEBOOK_COMMENTS_TOPICS = None
RENAME_TOPIC_TO_CLASS = False

In [ ]:
cur_run_params = run_params.general.create(ARTIFACTS_ENVIRONMENT_KEY, TENANT_ID, RUN_DATETIME)
    
url_config = {
    "YEAR_FILTER": int(YEAR_FILTER),
    "MONTH_FILTER": int(MONTH_FILTER),
    "OBJECT_TYPE": OBJECT_TYPE
}

TAGGING_RUNS_URL_TOPICS = cur_run_params.art_url_reg.get_url("tagging_runs-topics", url_config)
TAGGING_RUNS_URL_FACEBOOK_POSTS_TOPICS_FINAL = cur_run_params.art_url_reg.get_url("tagging_runs-facebook_posts_topics_final", url_config)
TAGGING_RUNS_URL_FACEBOOK_COMMENTS_FINAL = cur_run_params.art_url_reg.get_url("tagging_runs-facebook_comments_final", url_config)
TAGGING_RUNS_URL_FACEBOOK_COMMENTS_TOPICS_FINAL = cur_run_params.art_url_reg.get_url("tagging_runs-facebook_comments_topics_final", url_config)
if not FINAL_URL_FACEBOOK_COMMENTS_TOPICS and not RENAME_TOPIC_TO_CLASS:
    FINAL_URL_FACEBOOK_COMMENTS_TOPICS = cur_run_params.art_url_reg.get_url("final-facebook_comments_topics", url_config)

if not FINAL_URL_FACEBOOK_COMMENTS_TOPICS and RENAME_TOPIC_TO_CLASS:
    FINAL_URL_FACEBOOK_COMMENTS_TOPICS = cur_run_params.art_url_reg.get_url("final-facebook_comments_classes", url_config)

In [ ]:
# Display params.
print(
TAGGING_RUNS_URL_TOPICS,
TAGGING_RUNS_URL_FACEBOOK_POSTS_TOPICS_FINAL,
TAGGING_RUNS_URL_FACEBOOK_COMMENTS_FINAL,
TAGGING_RUNS_URL_FACEBOOK_COMMENTS_TOPICS_FINAL,
FINAL_URL_FACEBOOK_COMMENTS_TOPICS,
cur_run_params.run_dt.dt,
sep='\n',
)

In [ ]:
facebook_comments_topics_df = artifacts.dataframes.get(TAGGING_RUNS_URL_FACEBOOK_COMMENTS_FINAL).dataframe

In [ ]:
facebook_comments_topics_df.head()

In [ ]:
facebook_comments_topics_df.shape

In [ ]:
facebook_posts_topics_df = artifacts.dataframes.get(TAGGING_RUNS_URL_FACEBOOK_POSTS_TOPICS_FINAL).dataframe

In [ ]:
facebook_posts_topics_df

In [ ]:
facebook_posts_topics_df.shape

In [ ]:
facebook_comments_topics_final = finalise.inherit_facebook_comment_topics_from_facebook_posts_topics_df(
    facebook_posts_topics_df, facebook_comments_topics_df, True, ["topic", "has_topic", "features", "features_count"], rename_topic_to_class=RENAME_TOPIC_TO_CLASS
)

In [ ]:
facebook_comments_topics_final.head()

In [ ]:
facebook_comments_topics_final.shape

In [ ]:
facebook_comments_topics_final.dtypes

In [ ]:
_ = artifacts.dataframes.persist(FINAL_URL_FACEBOOK_COMMENTS_TOPICS, facebook_comments_topics_final)

In [ ]:
_ = artifacts.dataframes.persist(TAGGING_RUNS_URL_FACEBOOK_COMMENTS_TOPICS_FINAL, facebook_comments_topics_final)

In [ ]:
artifacts.dataframes.read_schema(TAGGING_RUNS_URL_FACEBOOK_COMMENTS_TOPICS_FINAL)